In [ ]:
print(data.columns)

data_cluster = data[['TotalValue']]
data_cluster = data_cluster.dropna()

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_cluster)

inertia = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), inertia, marker='o', color='b', linestyle='--')
plt.title('Elbow Method for Optimal Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()

optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data_cluster['Cluster'] = kmeans.fit_predict(data_scaled)

db_index = davies_bouldin_score(data_scaled, data_cluster['Cluster'])
print(f'Davies-Bouldin Index: {db_index}')

pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)
data_cluster['PCA1'] = data_pca[:, 0]
data_cluster['PCA2'] = data_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', palette='viridis', data=data_cluster, s=100, alpha=0.7)
plt.title(f'Customer Segments (K={optimal_k})')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Cluster')
plt.show()

cluster_centers = kmeans.cluster_centers_
print(f'Cluster Centers:\n{cluster_centers}')

for cluster_num in range(optimal_k):
    print(f"\nCluster {cluster_num} Summary:")
    print(data_cluster[data_cluster['Cluster'] == cluster_num].describe())

data_cluster.to_csv('Customer_Segments.csv', index=False)
